# QUANTITATIVE MOMENTUM STRATEGY

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, I've build an investing strategy that selects the n-stocks with the highest price momentum. Further calculating the recommended trades for an equal-weight portfolio of these n-stocks.

Momentum investing is a trading strategy in which investors buy securities that are rising and sell them when they look to have peaked. The strategy is to filter out highest momentum stocks i.e., stocks which are stable and perform well in long periods of time.

* **High-quality momentum stocks** perform slow and steady over long periods of time and are significantly less volatile.

* **Low-quality momentum stocks** are ones that does not show any momentum for a long time.

High-quality momentum stocks are  preferred because low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future.

To identify high-quality momentum, I've built a strategy that selects stocks from the highest percentiles of:

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns
* 5-year price returns


# Importing Libraries

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import stats
from scipy.stats import percentileofscore as pctile
from statistics import mean

# Requests library is extremely useful for HTTPS requests in Python
import requests

In [45]:
# To extract data from the Cloud API

!pip install iexfinance
from iexfinance.stocks import Stock

# Data Extraction

In [46]:
from secrets import IEX_CLOUD_API_TOKEN

# List of Stock data required
stocks=pd.read_csv('sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Batch API calls

In [42]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

print(symbol_groups)

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])


[0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
      ...  
95     CINF
96       CL
97      CLX
98      CMA
99    CMCSA
Name: Ticker, Length: 100, dtype: object, 100     CME
101     CMG
102     CMI
103     CMS
104     CNC
       ... 
195    FTNT
196     FTV
197      GD
198      GE
199    GILD
Name: Ticker, Length: 100, dtype: object, 200     GIS
201      GL
202     GLW
203      GM
204    GOOG
       ... 
295     MAA
296     MAR
297     MAS
298     MCD
299    MCHP
Name: Ticker, Length: 100, dtype: object, 300     MCK
301     MCO
302    MDLZ
303     MDT
304     MET
       ... 
395     RHI
396     RJF
397      RL
398     RMD
399     ROK
Name: Ticker, Length: 100, dtype: object, 400     ROL
401     ROP
402    ROST
403     RSG
404     RTX
       ... 
495    XLNX
496     XOM
497    XRAY
498     XRX
499     XYL
Name: Ticker, Length: 100, dtype: object, 500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, dtype: object]


# Momentum Percentiles

To calculate momentum percentile scores for every stock in the universe, we calculate percentile scores for the following metrics for every stock:

* Five-Year Price Return
* One-Year Price Return
* Six-Month Price Return
* Three-Month Price Return
* One-Month Price Return

In [47]:
# Creates a list of columns for dataframe required to calculate and compare HQM stocks

HQM_cols= [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'Five-Year Price Return', 
                'Five-Year Return Percentile',
           
                'One-Year Price Return', 
                'One-Year Return Percentile',
           
                'Six-Month Price Return',
                'Six-Month Return Percentile',
           
                'Three-Month Price Return',
                'Three-Month Return Percentile',
           
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
]

# Dataframe that contains data required for calculation and analysis of stocks based on their momentum
hqm_df = pd.DataFrame(columns= HQM_cols)
for symbol_string in symbol_strings:
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        if symbol in data.keys():
            hqm_df=hqm_df.append(
                pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['stats']['year5ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index=HQM_cols),
                ignore_index=True
            )

C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append metho

C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append metho

C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append metho

C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append metho

C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append metho

...

C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append metho

C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append metho

C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append metho

C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append metho

C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df=hqm_df.append(
C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\1814842518.py:32: FutureWarning: The frame.append metho

In [48]:
data['YUM']['stats']['year1ChangePercent']

-0.12010239132154668

# Data Preprocessing: Removing stocks with missing values

In [49]:
x= hqm_df[['Ticker', 'Five-Year Price Return', 'One-Year Price Return','Six-Month Price Return','Three-Month Price Return','One-Month Price Return',]].copy()
x[x.isnull().any(axis=1)].index
# rv_df[rv_df.isnull().any(axis=1)]

Int64Index([118, 164, 324, 325], dtype='int64')

In [50]:
# x.T['CTL']
# x.loc[x['Ticker'] == 'CTL']
x.drop([0, 1])

,Ticker,Five-Year Price Return,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return
2,AAP,1.361134,0.012854,-0.039058,0.153724,0.075609
3,AAPL,3.641085,0.165652,0.028454,0.25684,0.128202
4,ABBV,1.566378,0.249511,0.002603,-0.052599,-0.041097
5,ABC,1.127844,0.290554,0.083411,0.015658,0.079715
6,ABMD,0.790661,-0.226877,-0.094064,0.150303,-0.019867
...,...,...,...,...,...,...
496,YUM,0.686703,-0.120102,-0.070876,0.048456,-0.034914
497,ZBH,0.056104,-0.218616,-0.063012,-0.035765,0.050245
498,ZBRA,2.226314,-0.431932,-0.220098,0.009376,0.009369
499,ZION,0.245708,0.098077,-0.182126,0.120497,0.089482


In [51]:
# for i in x[x.isnull().any(axis=1)].index:
hqm_df.drop(x[x.isnull().any(axis=1)].index, inplace= True)

In [54]:
hqm_df.reset_index(drop=True, inplace=True)

In [55]:
hqm_df['Five-Year Price Return'][118]

0.0433374343761203

In [56]:
t_periods=['Five-Year' ,
           'One-Year',
          'Six-Month',
          'Three-Month',
          'One-Month']

from scipy.stats import percentileofscore as ptile

for row in hqm_df.index:
    for pd in t_periods:
        change_col=f'{pd} Price Return'
        pct_col=f'{pd} Return Percentile'
#         hqm_df.loc[row, pct_col]=stats.percentileofscore(hqm_df[change_col].tolist(), hqm_df.loc[row, change_col])
        hqm_df.loc[row, f'{pd} Return Percentile'] = ptile(hqm_df[f'{pd} Price Return'], hqm_df.loc[row, f'{pd} Price Return'])/100

for pd in t_periods:
    print(hqm_df[f'{pd} Return Percentile'])

print('\nFinal Dataframe\n')
hqm_df

0       0.77666
1      0.006036
2      0.768612
3      0.977867
4      0.822938
         ...   
492    0.484909
493    0.177062
494    0.915493
495    0.255533
496    0.889336
Name: Five-Year Return Percentile, Length: 497, dtype: object
0      0.247485
1      0.126761
2      0.571429
3      0.782696
4      0.855131
         ...   
492    0.331992
493    0.167002
494    0.038229
495    0.706237
496    0.241449
Name: One-Year Return Percentile, Length: 497, dtype: object
0      0.653924
1        0.1167
2      0.460765
3       0.62173
4      0.557344
         ...   
492    0.364185
493    0.390342
494    0.104628
495    0.171026
496    0.287726
Name: Six-Month Return Percentile, Length: 497, dtype: object
0      0.651911
1      0.050302
2      0.806841
3      0.937626
4      0.134809
         ...   
492    0.450704
493    0.160966
494    0.301811
495    0.684105
496    0.440644
Name: Three-Month Return Percentile, Length: 497, dtype: object
0      0.895372
1      0.036217
2      0.547284

,Ticker,Price,Number of Shares to Buy,Five-Year Price Return,Five-Year Return Percentile,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.18,N/A,1.385394,0.77666,-0.178047,0.247485,0.047163,0.653924,0.10407,0.651911,0.151014,0.895372,N/A
1,AAL,14.88,N/A,-0.697015,0.006036,-0.243699,0.126761,-0.213657,0.1167,-0.134048,0.050302,-0.069916,0.036217,N/A
2,AAP,211.82,N/A,1.361134,0.768612,0.012854,0.571429,-0.039058,0.460765,0.153724,0.806841,0.075609,0.547284,N/A
3,AAPL,177.2,N/A,3.641085,0.977867,0.165652,0.782696,0.028454,0.62173,0.25684,0.937626,0.128202,0.802817,N/A
4,ABBV,146.27,N/A,1.566378,0.822938,0.249511,0.855131,0.002603,0.557344,-0.052599,0.134809,-0.041097,0.066398,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,120.72,N/A,0.686703,0.484909,-0.120102,0.331992,-0.070876,0.364185,0.048456,0.450704,-0.034914,0.078471,N/A
493,ZBH,117.46,N/A,0.056104,0.177062,-0.218616,0.167002,-0.063012,0.390342,-0.035765,0.160966,0.050245,0.396378,N/A
494,ZBRA,335.62,N/A,2.226314,0.915493,-0.431932,0.038229,-0.220098,0.104628,0.009376,0.301811,0.009369,0.221328,N/A
495,ZION,60.49,N/A,0.245708,0.255533,0.098077,0.706237,-0.182126,0.171026,0.120497,0.684105,0.089482,0.627767,N/A


# High Quality Momentum Score for each stock

HQM Score is calculated to filter stocks in the investing strategy.

The HQM Score is the arithmetic mean of the 5 momentum percentile scores that we calculated in the last section.

In [57]:
for i in hqm_df.index:
    momentum_percentile=[]

    for pd in t_periods:
        momentum_percentile.append(hqm_df.loc[i, f'{pd} Return Percentile'])
    hqm_df.loc[i, 'HQM Score']=mean(momentum_percentile)

# The number of shares to be invested

Strategy: Investing in top n-high quality momentum stocks

## Selection of n-best momentum stocks

Accepts the number of stocks the investor is wants to invest in, identifying the 'n' best momentum stocks in our universe by sorting the DataFrame on the HQM Score column and dropping all but the top 'n' entries.

Further, removing the remaining Low-Momentum Stocks


In [59]:
def no_of_stocks():
    n = input("Enter the number of stocks you would like to invest in (less than or equal to 500):")

    try:
        value = float(n)
    except ValueError:                                       
       # If the user does not enter a number
        print("This is not a number! \n Please try again:")
        n = input("Enter the number of stocks you would like to invest in (less than or equal to 500):")

    return(n)

n = no_of_stocks()

hqm_df.sort_values(by='HQM Score', ascending= False, inplace=True)
final_hqm_df=hqm_df[:int(n)]
final_hqm_df.reset_index(drop=True, inplace=True )

Enter the number of stocks you would like to invest in (less than or equal to 500):50


## Portfolio Size

Portfolio size contains the total amount the investor is willing to invest.

In [60]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        value = float(portfolio_size)
    except ValueError:                                       
      # If the user does not enter the numericals
        print("This is not a number! \n Please try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

amt=float(portfolio_size)/len(final_hqm_df.index)
for i in range(len(final_hqm_df['Ticker'])):
    final_hqm_df.loc[i, 'Number of Shares to Buy']=math.floor(amt/final_hqm_df['Price'][i])


Enter the value of your portfolio:100000
100000


C:\Users\smrit\AppData\Local\Temp\ipykernel_7228\269478390.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_hqm_df.loc[i, 'Number of Shares to Buy']=math.floor(amt/final_hqm_df['Price'][i])


In [61]:
final_hqm_df.sort_values(by='Number of Shares to Buy', ascending = False).reset_index(drop = True)

,Ticker,Price,Number of Shares to Buy,Five-Year Price Return,Five-Year Return Percentile,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AIV,9.58,208,0.795588,0.559356,0.414843,0.927565,0.332899,0.975855,0.572767,1.0,0.26275,0.993964,0.891348
1,AES,26.6,75,1.786892,0.873239,0.082942,0.688129,0.223758,0.911469,0.266139,0.947686,0.291553,0.997988,0.883702
2,UNM,40.94,48,0.006872,0.158954,0.638724,0.959759,0.417697,0.987928,0.173196,0.865191,0.219082,0.967807,0.787928
3,HRB,48.32,41,1.028936,0.655936,0.997537,0.983903,0.963805,1.0,0.456537,0.997988,0.267912,0.995976,0.926761
4,IRM,55.32,36,1.099826,0.690141,0.276263,0.879276,0.311704,0.963783,0.12809,0.712274,0.188784,0.95171,0.839437
5,DRE,64.83,30,1.623901,0.843058,0.272734,0.875252,0.216314,0.901408,0.254967,0.93159,0.085021,0.603622,0.830986
6,DVN,70.94,28,1.686141,0.855131,1.889467,0.995976,0.304569,0.959759,-0.004071,0.247485,0.196476,0.955734,0.802817
7,CAH,70.22,28,0.295542,0.283702,0.406448,0.923541,0.313103,0.965795,0.260476,0.94165,0.25118,0.987928,0.820523
8,SO,83.15,24,1.069965,0.674044,0.24898,0.853119,0.282649,0.945674,0.101693,0.639839,0.14473,0.869215,0.796378
9,LW,82.96,24,0.931812,0.613682,0.282,0.881288,0.249252,0.923541,0.315743,0.981891,0.102607,0.684105,0.816901


# Formatting the output in excel

Converting the recommended shares based on momentum of stocks to an excel file.

In [ ]:
final_hqm_df.to_excel('Recommended_Shares_based_on_momentum.xlsx')